<a href="https://colab.research.google.com/github/HasibAlMuzdadid/Convolutional-Neural-Network/blob/main/convolutional_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Convolutional Neural Network**

In [ ]:
import numpy as np
import h5py
import matplotlib.pyplot as plt

A convolution layer transforms an input volume into an output volume of different size.

we will build every step of the convolution layer. We will first implement two helper functions: one for zero padding and the other for computing the convolution function itself. 

**Zero Padding**

Zero padding adds zeros around the border of an image.

- It allows us to use a CONV layer without necessarily shrinking the height and width of the volumes. This is important for building deeper networks, since otherwise the height/width would shrink as we go to deeper layers. An important special case is the "same" convolution, in which the height/width is exactly preserved after one layer. 

- It helps us keep more of the information at the border of an image. Without padding, very few values at the next layer would be affected by pixels at the edges of an image.


In [ ]:
def zero_pad(X, pad):
    
    # X -> python numpy array of shape (m, n_H, n_W, n_C) representing a batch of m images
    # pad -> integer, amount of padding around each image on vertical and horizontal dimensions
    # X_pad -> padded image of shape (m, n_H + 2 * pad, n_W + 2 * pad, n_C)

    X_pad = np.pad(X, ((0, 0), (pad, pad), (pad, pad), (0, 0)), "constant", constant_values=(0, 0))
    
    return X_pad

**Single Step of Convolution**

We will implement a single step of convolution, in which we will apply the filter to a single position of the input. This will be used to build a convolutional unit, which: 

- Takes an input volume 
- Applies a filter at every position of the input
- Outputs another volume (usually of different size)

We will convolve a 3x3 filter with the image by multiplying its values element-wise with the original matrix, then summing them up and adding a bias. In this first step of the exercise, we will implement a single step of convolution, corresponding to applying a filter to just one of the positions to get a single real-valued output. 

Later in this notebook, we'll apply this function to multiple positions of the input to implement the full convolutional operation.


In [ ]:
# single step convolution

def conv_single_step(a_slice_prev, W, b):
    
    # a_slice_prev -> slice of input data of shape (f, f, n_C_prev)
    # W -> Weight parameters contained in a window - matrix of shape (f, f, n_C_prev)
    # b -> Bias parameters contained in a window - matrix of shape (1, 1, 1)
    # Z -> a scalar value, the result of convolving the sliding window (W, b) on a slice x of the input data
  
    s = a_slice_prev * W                        # Element-wise product between a_slice_prev and W   
    Z = np.sum(s)                               # Sum over all entries of the volume s  
    Z = Z + float(b)                            # Add bias b to Z. Cast b to a float() so that Z results in a scalar value

    return Z

**Convolutional Neural Network - Forward Pass**

We will take many filters and convolve them on the input. Each 'convolution' gives a 2D matrix output. We will then stack these outputs to get a 3D volume.

The formulas relating the output shape of the convolution to the input shape are:
    
$$n_H = \Bigl\lfloor \frac{n_{H_{prev}} - f + 2 \times pad}{stride} \Bigr\rfloor +1$$
$$n_W = \Bigl\lfloor \frac{n_{W_{prev}} - f + 2 \times pad}{stride} \Bigr\rfloor +1$$
$$n_C = \text{number of filters used in the convolution}$$

In [ ]:
# forward convolution

def conv_forward(A_prev, W, b, hparameters):
   
    # A_prev -> output activations of the previous layer, numpy array of shape (m, n_H_prev, n_W_prev, n_C_prev)
    # W -> Weights, numpy array of shape (f, f, n_C_prev, n_C)
    # b -> Biases, numpy array of shape (1, 1, 1, n_C)
    # hparameters -> python dictionary containing "stride" and "pad"
    # Z -> conv output, numpy array of shape (m, n_H, n_W, n_C)
    # cache -> cache of values needed for the conv_backward() function

  
    (m, n_H_prev, n_W_prev, n_C_prev) = np.shape(A_prev)                      # Retrieve dimensions from A_prev's shape
    (f, f, n_C_prev, n_C) = np.shape(W)                                       # Retrieve dimensions from W's shape
    
    stride = hparameters['stride']                                            # Retrieve information from "hparameters"
    pad = hparameters['pad']
    
    n_H = int((n_H_prev - f + 2 * pad) / stride) + 1                          # Compute the dimensions of the CONV output volume using the formula 
    n_W = int((n_W_prev - f + 2 * pad) / stride) + 1                          # Use int() to apply the 'floor' operation
    
    Z = np.zeros((m, n_H, n_W, n_C))                                          # Initialize the output volume Z with zeros
    
    A_prev_pad = zero_pad(A_prev, pad)                                        # Create A_prev_pad by padding A_prev
    
    for i in range(m):                                                        # loop over the batch of training examples
        a_prev_pad = A_prev_pad[i,:,:,:]                                      # Select ith training example's padded activation
        
        for h in range(n_H):                                                  # loop over vertical axis of the output volume                                                                  
            vert_start = h * stride                                           # Find the vertical start and end of the current "slice"
            vert_end = h * stride+ f
            
            for w in range(n_W):                                              # loop over horizontal axis of the output volume
                horiz_start = w * stride                                      # Find the horizontal start and end of the current "slice"
                horiz_end = w * stride + f
                
                for c in range(n_C):                                          # loop over channels (= #filters) of the output volume
                    a_slice_prev = a_prev_pad[vert_start:vert_end,horiz_start:horiz_end,:]         # Use the corners to define the (3D) slice of a_prev_pad 
                                       
                    weights = W[:, :, :, c]                                   # Convolve the (3D) slice with the correct filter W and bias b, to get back one output neuron
                    biases  = b[:, :, :, c]
                    Z[i, h, w, c] = conv_single_step(a_slice_prev, weights, biases)
    
    
    cache = (A_prev, W, b, hparameters)                                       # Save information in "cache" for the backprop
    
    
    return Z, cache

**Pooling Layer** 

The pooling (POOL) layer reduces the height and width of the input. It helps in reducing computation, as well as helps to make feature detectors more invariant to its position in the input. The two types of pooling layers are: 

- Max-pooling layer : slides an ($f, f$) window over the input and stores the max value of the window in the output.

- Average-pooling layer : slides an ($f, f$) window over the input and stores the average value of the window in the output.

These pooling layers have no parameters for backpropagation to train. However, they have hyperparameters such as the window size $f$. This specifies the height and width of the $f \times f$ window we would compute a *max* or *average* over. 

the formulas binding the output shape of the pooling to the input shape are:

$$n_H = \Bigl\lfloor \frac{n_{H_{prev}} - f}{stride} \Bigr\rfloor +1$$

$$n_W = \Bigl\lfloor \frac{n_{W_{prev}} - f}{stride} \Bigr\rfloor +1$$

$$n_C = n_{C_{prev}}$$

In [ ]:
# forward pooling

def pool_forward(A_prev, hparameters, mode = "max"):
    
    # A_prev -> Input data, numpy array of shape (m, n_H_prev, n_W_prev, n_C_prev)
    # hparameters -> python dictionary containing "f" and "stride"
    # mode -> the pooling mode we would like to use, defined as a string ("max" or "average")
    # A -> output of the pool layer, a numpy array of shape (m, n_H, n_W, n_C)
    # cache -> cache used in the backward pass of the pooling layer, contains the input and hparameters 

    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape                        # Retrieve dimensions from the input shape
    
    f = hparameters["f"]                                                    # Retrieve hyperparameters from "hparameters"
    stride = hparameters["stride"]
    
    n_H = int(1 + (n_H_prev - f) / stride)                                  # Define the dimensions of the output
    n_W = int(1 + (n_W_prev - f) / stride)
    n_C = n_C_prev
    
    A = np.zeros((m, n_H, n_W, n_C))                                        # Initialize output matrix A   
    
   
    for i in range(m):                                                      # loop over the training examples

        for h in range(n_H):                                                # loop on the vertical axis of the output volume
            vert_start = stride * h                                         # Find the vertical start and end of the current "slice"
            vert_end = vert_start + f
            
            for w in range(n_W):                                            # loop on the horizontal axis of the output volume
                horiz_start = stride * w                                    # Find the horizontal start and end of the current "slice"
                horiz_end = horiz_start + f
                
                for c in range (n_C):                                       # loop over the channels of the output volume
                    a_prev_slice = A_prev[i, vert_start:vert_end, horiz_start:horiz_end,c]       # Use the corners to define the current slice on the ith training example of A_prev, channel c
                    
                    if mode == "max":                                       # Compute the pooling operation on the slice
                        A[i, h, w, c] = np.max(a_prev_slice)
                    elif mode == "average":
                        A[i, h, w, c] = np.mean(a_prev_slice)
    
    
    cache = (A_prev, hparameters)                                           # Store the input and hparameters in "cache" for pool_backward()
    
    return A, cache

**Backpropagation in Convolutional Neural Network**

***Computing dA:***

This is the formula for computing $dA$ with respect to the cost for a certain filter $W_c$ and a given training example:

$$dA \mathrel{+}= \sum _{h=0} ^{n_H} \sum_{w=0} ^{n_W} W_c \times dZ_{hw} \tag{1}$$

Where, $W_c$ is a filter and $dZ_{hw}$ is a scalar corresponding to the gradient of the cost with respect to the output of the conv layer Z at the hth row and wth column (corresponding to the dot product taken at the ith stride left and jth stride down). Note that at each time, we multiply the the same filter $W_c$ by a different dZ when updating dA. We do so mainly because when computing the forward propagation, each filter is dotted and summed by a different a_slice. Therefore when computing the backprop for dA, we are just adding the gradients of all the a_slices. 



In code, inside the appropriate for-loops, this formula translates into:
```python
da_prev_pad[vert_start:vert_end, horiz_start:horiz_end, :] += W[:,:,:,c] * dZ[i, h, w, c]
```

***Computing dW:***

This is the formula for computing $dW_c$ ($dW_c$ is the derivative of one filter) with respect to the loss:

$$dW_c  \mathrel{+}= \sum _{h=0} ^{n_H} \sum_{w=0} ^ {n_W} a_{slice} \times dZ_{hw}  \tag{2}$$

Where $a_{slice}$ corresponds to the slice which was used to generate the activation $Z_{ij}$. Hence, this ends up giving us the gradient for $W$ with respect to that slice. Since it is the same $W$, we will just add up all such gradients to get $dW$. 



In code, inside the appropriate for-loops, this formula translates into:
```python
dW[:,:,:,c] \mathrel{+}= a_slice * dZ[i, h, w, c]
```

***Computing db:***

This is the formula for computing $db$ with respect to the cost for a certain filter $W_c$:

$$db = \sum_h \sum_w dZ_{hw} \tag{3}$$

As we know in basic neural networks, db is computed by summing $dZ$. In this case, we are just summing over all the gradients of the conv output (Z) with respect to the cost. 


In code, inside the appropriate for-loops, this formula translates into:
```python
db[:,:,:,c] += dZ[i, h, w, c]
```


In [ ]:
# backword convolution

def conv_backward(dZ, cache):

    # dZ -> gradient of the cost with respect to the output of the conv layer (Z), numpy array of shape (m, n_H, n_W, n_C)
    # cache -> cache of values needed for the conv_backward(), output of conv_forward()
    # 
    # Returns:
    # dA_prev -> gradient of the cost with respect to the input of the conv layer (A_prev), numpy array of shape (m, n_H_prev, n_W_prev, n_C_prev)
    # dW -> gradient of the cost with respect to the weights of the conv layer (W), numpy array of shape (f, f, n_C_prev, n_C)
    # db -> gradient of the cost with respect to the biases of the conv layer (b), numpy array of shape (1, 1, 1, n_C)
    
    (A_prev, W, b, hparameters) = cache                                   # Retrieve information from "cache"
   
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape                      # Retrieve dimensions from A_prev's shape
   
    (f, f, n_C_prev, n_C) = W.shape                                       # Retrieve dimensions from W's shape
    
    stride = hparameters["stride"]                                        # Retrieve information from "hparameters"
    pad = hparameters["pad"]
    
    (m, n_H, n_W, n_C) = dZ.shape                                         # Retrieve dimensions from dZ's shape
    
    dA_prev = np.zeros(A_prev.shape)                                      # Initialize dA_prev, dW, db with the correct shapes            
    dW = np.zeros(W.shape)
    db = np.zeros(b.shape)                                                # b.shape = [1,1,1,n_C]
    
    A_prev_pad = zero_pad(A_prev, pad)                                    # Pad A_prev and dA_prev
    dA_prev_pad = zero_pad(dA_prev, pad)
    
    for i in range(m):                                                    # loop over the training examples           
        a_prev_pad = A_prev_pad[i]                                        # select ith training example from A_prev_pad and dA_prev_pad
        da_prev_pad = dA_prev_pad[i]
        
        for h in range(n_H):                                              # loop over vertical axis of the output volume         
            for w in range(n_W):                                          # loop over horizontal axis of the output volume
                for c in range(n_C):                                      # loop over the channels of the output volume
                    
                    vert_start = stride * h                               # Find the corners of the current "slice"
                    vert_end = vert_start + f
                    horiz_start = stride * w
                    horiz_end = horiz_start + f
                    
                    a_slice = a_prev_pad[vert_start:vert_end,horiz_start:horiz_end,:]          # Use the corners to define the slice from a_prev_pad

                    da_prev_pad[vert_start:vert_end, horiz_start:horiz_end, :] += W[:,:,:,c] * dZ[i, h, w, c]       # Update gradients for the window and the filter's parameters using the code formulas given above
                    dW[:,:,:,c] += a_slice * dZ[i, h, w, c]
                    db[:,:,:,c] += dZ[i, h, w, c]
    
        dA_prev[i, :, :, :] = da_prev_pad[pad:-pad, pad:-pad, :]          # Set the ith training example's dA_prev to the unpadded da_prev_pad

    
    return dA_prev, dW, db

**Pooling Layer - Backward Pass**

Even though a pooling layer has no parameters for backprop to update, we still need to backpropagate the gradient through the pooling layer in order to compute gradients for layers that came before the pooling layer.

***Max Pooling - Backward Pass*** 

Before jumping into the backpropagation of the pooling layer, we are going to build a helper function called `create_mask_from_window()` which does the following: 

$$ X = \begin{bmatrix}
1 && 3 \\
4 && 2
\end{bmatrix} \quad \rightarrow  \quad M =\begin{bmatrix}
0 && 0 \\
1 && 0
\end{bmatrix}\tag{4}$$

As we can see, this function creates a "mask" matrix which keeps track of where the maximum of the matrix is. True (1) indicates the position of the maximum in X, the other entries are False (0). 

- If you have a matrix X and a scalar x : `A = (X == x)` will return a matrix A of the same size as X such that:
```
A[i,j] = True if X[i,j] = x
A[i,j] = False if X[i,j] != x
```

In [ ]:
def create_mask_from_window(x):

    # x -> Array of shape (f, f)
    # mask -> Array of the same shape as window, contains a True at the position corresponding to the max entry of x.

    mask = (x == np.max(x))

    return mask

***Average Pooling - Backward Pass*** 

In max pooling, for each input window, all the "influence" on the output came from a single input value--the max. In average pooling, every element of the input window has equal influence on the output. So to implement backprop, we will now implement a helper function that reflects this.

For example if we did average pooling in the forward pass using a 2x2 filter, then the mask we'll use for the backward pass will look like: 
$$ dZ = 1 \quad \rightarrow  \quad dZ =\begin{bmatrix}
1/4 && 1/4 \\
1/4 && 1/4
\end{bmatrix}\tag{5}$$

This implies that each position in the $dZ$ matrix contributes equally to output because in the forward pass, we took an average.

In [ ]:
def distribute_value(dz, shape):
   
    # dz -> input scalar
    # shape -> the shape (n_H, n_W) of the output matrix for which we want to distribute the value of dz
    # a -> Array of size (n_H, n_W) for which we distributed the value of dz
 
    (n_H, n_W) = shape                                  # Retrieve dimensions from shape
    average = np.prod(shape)                            # Compute the value to distribute on the matrix 
    a = (dz/average)*np.ones(shape)                     # Create a matrix where every entry is the "average" value

    return a

**Putting It Together: Pooling Backward**




In [ ]:
# backward pass of the pooling layer

def pool_backward(dA, cache, mode = "max"):
    
    # dA -> gradient of cost with respect to the output of the pooling layer, same shape as A
    # cache -> cache output from the forward pass of the pooling layer, contains the layer's input and hparameters 
    # mode -> the pooling mode we would like to use, defined as a string ("max" or "average")
    # dA_prev -> gradient of cost with respect to the input of the pooling layer, same shape as A_prev

    (A_prev, hparameters) = cache                                # Retrieve information from cache
    
    stride = hparameters["stride"]                               # Retrieve hyperparameters from "hparameters"
    f = hparameters["f"]
    
    m, n_H_prev, n_W_prev, n_C_prev = A_prev.shape               # Retrieve dimensions from A_prev's shape and dA's shape
    m, n_H, n_W, n_C = dA.shape
    
    dA_prev = np.zeros(A_prev.shape)                             # Initialize dA_prev with zeros
    
    for i in range(m):                                           # loop over the training examples
        a_prev = A_prev[i,:,:,:]                                 # select training example from A_prev
        
        for h in range(n_H):                                     # loop on the vertical axis
            for w in range(n_W):                                 # loop on the horizontal axis
                for c in range(n_C):                             # loop over the channels (depth)
                    
                    vert_start  = h * stride                     # Find the corners of the current "slice"
                    vert_end    = h * stride + f
                    horiz_start = w * stride
                    horiz_end   = w * stride + f
                    
                    # Compute the backward propagation in both modes
                    if mode == "max":
                                               
                        a_prev_slice = a_prev[ vert_start:vert_end, horiz_start:horiz_end, c ]                 # Use the corners and "c" to define the current slice from a_prev                    
                        mask = create_mask_from_window( a_prev_slice )                                         # Create the mask from a_prev_slice
                        dA_prev[i, vert_start:vert_end, horiz_start:horiz_end, c] += mask * dA[i, h, w, c]     # Set dA_prev to be dA_prev + (the mask multiplied by the correct entry of dA)
                        
                    elif mode == "average":
                                              
                        da = dA[i, h, w, c]                                                                           # Get the value da from dA
                        shape = (f,f)                                                                                 # Define the shape of the filter as fxf
                        dA_prev[i, vert_start: vert_end, horiz_start: horiz_end, c] += distribute_value(da, shape)    # Distribute it to get the correct slice of dA_prev. i.e. Add the distributed value of da
    
    return dA_prev